In [1]:
import pandas as pd

In [2]:
t_df = pd.read_pickle ('data/interim/X_technique_org.pkl')
g_df = pd.read_pickle ('data/interim/X_group_org.pkl')
y_df = pd.read_csv ('data/interim/y_cleaned.csv')

- Technique: Used Frequency

In [24]:
"""
requirements: technique feature df, label df (after splitting).
steps:
1. from label df: get technique frequencies from positive labels
2. add frequencies to technique feature df
"""

t_count = y_df[y_df['label'] == 1.0]['technique_ID'].value_counts(sort = 'ascending')
res_df = pd.merge(left = t_df, right = t_count, on = 'technique_ID', how = 'left', )
res_df.rename(columns={'count': 'input_techinique_popularity'}, inplace = True)

- Group: Interaction Count

In [29]:
"""
requirements: group feature df, label df (after splitting)
1. from label df: get group frequencies from positive labels. Retrieve (Group_ID - Count) table
2. add frequencies to group feature df based on Group_ID with the table in step 1
"""
g_count = y_df[y_df['label'] == 1.0]['group_ID'].value_counts()
res_df = pd.merge (left= g_df, right = g_count, on = 'group_ID', how = 'left')
res_df.rename (columns= {'count': 'input_group_interaction_rate'}, inplace= True)

In [30]:
res_df

,group_ID,input_group_software_id,input_group_description,input_group_interaction_rate
0,G0099,[s0434],APT-C-36 is a suspected South America espionag...,9.0
1,G0006,"[s0017, s0025, s0119, s0026, s0121, s0002, s00...",APT1 is a Chinese threat group that has been a...,23.0
2,G0005,"[s0040, s0015, s0003]",APT12 is a threat group that has been attribut...,5.0
3,G0023,[s0064],APT16 is a China-based threat group that has l...,1.0
4,G0025,[s0069],APT17 is a China-based threat group that has c...,2.0
...,...,...,...,...
131,G0044,"[s0501, s0013, s0141]",Winnti Group is a threat group with Chinese or...,6.0
132,G0102,"[s0552, s0534, s0521, s0154, s0575, s0024, s03...",Wizard Spider is a Russia-based financially mo...,46.0
133,G0128,[],ZIRCONIUM is a threat group operating out of C...,26.0
134,G0018,"[s0043, s0042, s0039, s0012, s0096, s0100, s0104]",admin@338 is a China-based cyber threat group....,12.0


- Group: Number of used Tactics types

In [117]:
pos_y = y_df[y_df['label'] == 1]
g_tactic = pd.merge (left = pos_y[['group_ID','technique_ID']], right = t_df[['technique_ID','input_technique_tactics']], 
                     how = 'left', on = 'technique_ID')
g_tactic_counts = g_tactic.explode('input_technique_tactics').groupby('group_ID', as_index=False).agg(list)
g_tactic_counts['tactic_counts'] =  g_tactic_counts['input_technique_tactics'].apply(lambda x: len(set(x)))

In [118]:
g_tactic_counts

,group_ID,technique_ID,input_technique_tactics,tactic_counts
0,G0001,"[T1583.002, T1583.003, T1560, T1584.005, T1001...","[resource_development, resource_development, c...",10
1,G0002,[T1027.001],[defense_evasion],1
2,G0003,"[T1557.002, T1557.002, T1587.001, T1585.001, T...","[collection, credential_access, resource_devel...",3
3,G0004,"[T1087.002, T1087.001, T1071.004, T1071.001, T...","[discovery, discovery, command_and_control, co...",12
4,G0005,"[T1568.003, T1203, T1566.001, T1204.002, T1102...","[command_and_control, execution, initial_acces...",3
...,...,...,...,...
128,G1009,"[T1087.001, T1587.001, T1190, T1562.004, T1105...","[discovery, resource_development, initial_acce...",7
129,G1011,"[T1583.001, T1585.002, T1585.001, T1203, T1589...","[resource_development, resource_development, r...",5
130,G1012,"[T1005, T1585.001, T1566.003, T1204.002]","[collection, resource_development, initial_acc...",4
131,G1013,"[T1071.001, T1059.003, T1546.003, T1546.003, T...","[command_and_control, execution, persistence, ...",6


- Group: Tactic list: full list of used tactics

In [119]:
pos_y = y_df[y_df['label'] == 1]
g_tactic = pd.merge (left = pos_y[['group_ID','technique_ID']], right = t_df[['technique_ID','input_technique_tactics']], 
                     how = 'left', on = 'technique_ID')
g_tactic_list = g_tactic.explode('input_technique_tactics').groupby('group_ID', as_index=False).agg(list)
# g_tactic_counts['tactic_counts'] =  g_tactic_counts['input_technique_tactics'].apply(lambda x: len(set(x)))
g_tactic_list

,group_ID,technique_ID,input_technique_tactics
0,G0001,"[T1583.002, T1583.003, T1560, T1584.005, T1001...","[resource_development, resource_development, c..."
1,G0002,[T1027.001],[defense_evasion]
2,G0003,"[T1557.002, T1557.002, T1587.001, T1585.001, T...","[collection, credential_access, resource_devel..."
3,G0004,"[T1087.002, T1087.001, T1071.004, T1071.001, T...","[discovery, discovery, command_and_control, co..."
4,G0005,"[T1568.003, T1203, T1566.001, T1204.002, T1102...","[command_and_control, execution, initial_acces..."
...,...,...,...
128,G1009,"[T1087.001, T1587.001, T1190, T1562.004, T1105...","[discovery, resource_development, initial_acce..."
129,G1011,"[T1583.001, T1585.002, T1585.001, T1203, T1589...","[resource_development, resource_development, r..."
130,G1012,"[T1005, T1585.001, T1566.003, T1204.002]","[collection, resource_development, initial_acc..."
131,G1013,"[T1071.001, T1059.003, T1546.003, T1546.003, T...","[command_and_control, execution, persistence, ..."


- Group: interaction count per Tactics.

In [82]:
"""
requirements: technique-tactics features, group features df, labels df
"""

pos_y = y_df[y_df['label'] == 1]
g_tactic = pd.merge (left = pos_y[['group_ID','technique_ID']], right = t_df[['technique_ID','input_technique_tactics']], 
                     how = 'left', on = 'technique_ID')

g_tactic = g_tactic.explode('input_technique_tactics')
tactic_freq_df = pd.get_dummies(g_tactic,prefix='input_group_tactic_freq', columns = ['input_technique_tactics']).groupby('group_ID', as_index=False).sum()
tactic_freq_df.drop (columns=['technique_ID'], inplace= True)
res_df = pd.merge (left = g_df, right = tactic_freq_df, on = 'group_ID', how = 'left')

In [81]:
res_df

,group_ID,input_group_software_id,input_group_description,input_group_tactic_freq_collection,input_group_tactic_freq_command_and_control,input_group_tactic_freq_credential_access,input_group_tactic_freq_defense_evasion,input_group_tactic_freq_discovery,input_group_tactic_freq_execution,input_group_tactic_freq_exfiltration,input_group_tactic_freq_impact,input_group_tactic_freq_initial_access,input_group_tactic_freq_lateral_movement,input_group_tactic_freq_persistence,input_group_tactic_freq_privilege_escalation,input_group_tactic_freq_reconnaissance,input_group_tactic_freq_resource_development
0,G0099,[s0434],APT-C-36 is a suspected South America espionag...,0.0,2.0,0.0,2.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
1,G0006,"[s0017, s0025, s0119, s0026, s0121, s0002, s00...",APT1 is a Chinese threat group that has been a...,5.0,0.0,1.0,2.0,6.0,1.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,5.0
2,G0005,"[s0040, s0015, s0003]",APT12 is a threat group that has been attribut...,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,G0023,[s0064],APT16 is a China-based threat group that has l...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,G0025,[s0069],APT17 is a China-based threat group that has c...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,G0044,"[s0501, s0013, s0141]",Winnti Group is a threat group with Chinese or...,0.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
132,G0102,"[s0552, s0534, s0521, s0154, s0575, s0024, s03...",Wizard Spider is a Russia-based financially mo...,2.0,1.0,4.0,10.0,7.0,7.0,2.0,1.0,5.0,5.0,7.0,7.0,0.0,2.0
133,G0128,[],ZIRCONIUM is a threat group operating out of C...,0.0,3.0,1.0,5.0,5.0,3.0,2.0,0.0,1.0,0.0,1.0,2.0,2.0,2.0
134,G0018,"[s0043, s0042, s0039, s0012, s0096, s0100, s0104]",admin@338 is a China-based cyber threat group....,0.0,0.0,0.0,1.0,7.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [75]:
tactic_freq_df

,technique_ID,input_group_tactic_freq_collection,input_group_tactic_freq_command_and_control,input_group_tactic_freq_credential_access,input_group_tactic_freq_defense_evasion,input_group_tactic_freq_discovery,input_group_tactic_freq_execution,input_group_tactic_freq_exfiltration,input_group_tactic_freq_impact,input_group_tactic_freq_initial_access,input_group_tactic_freq_lateral_movement,input_group_tactic_freq_persistence,input_group_tactic_freq_privilege_escalation,input_group_tactic_freq_reconnaissance,input_group_tactic_freq_resource_development
group_ID,,,,,,,,,,,,,,,
G0001,T1583.002T1583.003T1560T1584.005T1001.002T1005...,2,1,1,2,0,1,0,0,4,2,2,2,0,3
G0002,T1027.001,0,0,0,1,0,0,0,0,0,0,0,0,0,0
G0003,T1557.002T1557.002T1587.001T1585.001T1003.001T...,1,0,2,0,0,0,0,0,0,0,0,0,0,3
G0004,T1087.002T1087.001T1071.004T1071.001T1560T1560...,7,3,6,6,12,3,2,0,4,1,5,4,0,2
G0005,T1568.003T1203T1566.001T1204.002T1102.002,0,2,0,0,0,2,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
G1009,T1087.001T1587.001T1190T1562.004T1105T1027T158...,0,1,0,3,3,0,0,0,1,1,1,0,0,2
G1011,T1583.001T1585.002T1585.001T1203T1589.002T1566...,0,1,0,0,0,3,0,0,3,0,0,0,4,4
G1012,T1005T1585.001T1566.003T1204.002,1,0,0,0,0,1,0,0,1,0,0,0,0,1


In [41]:
import pandas as pd

# Example DataFrame with lists of strings in a column
data = {
    'user_id': [1, 2, 3],
    'list_column': [['A', 'B', 'C', 'A'], ['B', 'C', 'C'], ['A', 'B', 'B']]
}

user_df = pd.DataFrame(data)

# Explode the list_column to have one row per string in the list
user_df_exploded = user_df.explode('list_column')

# Get dummies for each string value and aggregate by user_id
frequency_df = pd.get_dummies(user_df_exploded['list_column']).groupby(level=0).sum()

# Concatenate the frequency DataFrame with the original DataFrame
result_df = pd.concat([user_df, frequency_df], axis=1)

print(result_df)


   user_id   list_column  A  B  C
0        1  [A, B, C, A]  2  1  1
1        2     [B, C, C]  0  1  2
2        3     [A, B, B]  1  2  0
